Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel $\rightarrow$ Restart) and then **run all cells** (in the menubar, select Cell $\rightarrow$ Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Lucas Zhao"
COLLABORATORS = ""

---

In [1]:
import pandas as pd
from scipy.stats import pearsonr, spearmanr
from sklearn.linear_model import LinearRegression
import numpy as np

file_path = "HSR.csv"
data = pd.read_csv(file_path)
data['Total Pulls'] = pd.to_numeric(data['Total Pulls'].str.replace(',', ''), errors='coerce')
data['Youtube Views'] = pd.to_numeric(data['Youtube Views'].str.replace(',', ''), errors='coerce')
data['time in between first appearence in story and banner (days)'] = pd.to_numeric(
    data['time in between first appearence in story and banner (days)'], errors='coerce'
)
data['# pulled'] = pd.to_numeric(data['# pulled'].str.replace(',', ''), errors='coerce')
data['estimated character revenue (in millions)'] = pd.to_numeric(data['estimated character revenue (in millions)'].str.replace(r'[\$,]', '', regex=True), errors='coerce')
data = data.dropna(subset=['Total Pulls', 'Youtube Views', 'time in between first appearence in story and banner (days)', '# pulled', 'estimated character revenue (in millions)'])
total_pulls_stats = data['Total Pulls'].describe()
youtube_views_stats = data['Youtube Views'].describe()
time_between_stats = data['time in between first appearence in story and banner (days)'].describe()
pulled_stats = data['# pulled'].describe()
revenue_stats = data['estimated character revenue (in millions)'].describe()
def calculate_correlation_and_r2(x_column, x_name):
    pearson_corr, pearson_pval = pearsonr(data['Total Pulls'], data[x_column])
    spearman_corr, spearman_pval = spearmanr(data['Total Pulls'], data[x_column])
    X = data[x_column].values.reshape(-1, 1)
    y = data['Total Pulls'].values
    regressor = LinearRegression()
    regressor.fit(X, y)
    r_squared = regressor.score(X, y)
    print(f"\nCorrelation Between Total Pulls and {x_name}:")
    print(f"Pearson Correlation: {round(pearson_corr, 3)}, p-value: {round(pearson_pval, 3)}")
    print(f"Spearman Correlation: {round(spearman_corr, 3)}, p-value: {round(spearman_pval, 3)}")
    print(f"R^2 (Coefficient of Determination): {round(r_squared, 3)}")

calculate_correlation_and_r2('Youtube Views', 'Youtube Views')
calculate_correlation_and_r2('time in between first appearence in story and banner (days)', 'Time Between First Appearance and Banner (days)')
calculate_correlation_and_r2('# pulled', '# Pulled')
calculate_correlation_and_r2('estimated character revenue (in millions)', 'Estimated Character Revenue (in millions)')



Correlation Between Total Pulls and Youtube Views:
Pearson Correlation: 0.262, p-value: 0.187
Spearman Correlation: 0.364, p-value: 0.062
R^2 (Coefficient of Determination): 0.069

Correlation Between Total Pulls and Time Between First Appearance and Banner (days):
Pearson Correlation: 0.231, p-value: 0.245
Spearman Correlation: 0.201, p-value: 0.316
R^2 (Coefficient of Determination): 0.054

Correlation Between Total Pulls and # Pulled:
Pearson Correlation: 0.991, p-value: 0.0
Spearman Correlation: 0.99, p-value: 0.0
R^2 (Coefficient of Determination): 0.981

Correlation Between Total Pulls and Estimated Character Revenue (in millions):
Pearson Correlation: 1.0, p-value: 0.0
Spearman Correlation: 1.0, p-value: 0.0
R^2 (Coefficient of Determination): 1.0


In [2]:
import pandas as pd
from scipy.stats import f_oneway

file_path = "HSR.csv"
data = pd.read_csv(file_path)
data['Total Pulls'] = pd.to_numeric(data['Total Pulls'].str.replace(',', ''), errors='coerce')
categorical_columns = ['Importance to Main Story', 'Role', 'Type', 'Gender', 'Path', 'Element']
data = data.dropna(subset=['Total Pulls'] + categorical_columns)
def analyze_categorical_relationship(data, categorical_column):
    print(f"\nAnalyzing {categorical_column}:")
    group_means = data.groupby(categorical_column)['Total Pulls'].mean()
    group_medians = data.groupby(categorical_column)['Total Pulls'].median()
    group_std = data.groupby(categorical_column)['Total Pulls'].std()
    groups = [data[data[categorical_column] == group]['Total Pulls'] for group in data[categorical_column].unique()]
    if len(groups) > 1:
        anova_result = f_oneway(*groups)
        print("\nANOVA Results:")
        print(f"F-statistic: {round(anova_result.statistic, 3)}, p-value: {round(anova_result.pvalue, 3)}")
    else:
        print("\nANOVA cannot be performed: Not enough valid groups.")
for column in categorical_columns:
    analyze_categorical_relationship(data, column)



Analyzing Importance to Main Story:

ANOVA Results:
F-statistic: 4.554, p-value: 0.013

Analyzing Role:

ANOVA Results:
F-statistic: 0.323, p-value: 0.809

Analyzing Type:

ANOVA Results:
F-statistic: 2.313, p-value: 0.152

Analyzing Gender:

ANOVA Results:
F-statistic: 2.31, p-value: 0.122

Analyzing Path:

ANOVA Results:
F-statistic: 1.055, p-value: 0.422

Analyzing Element:

ANOVA Results:
F-statistic: 1.691, p-value: 0.178


In [3]:
import sqlite3
import pandas as pd
from scipy.stats import pearsonr, spearmanr, f_oneway
from sklearn.linear_model import LinearRegression
import numpy as np

file_path = "HSR.csv"
data = pd.read_csv(file_path)
data['Total Pulls'] = pd.to_numeric(data['Total Pulls'].str.replace(',', ''), errors='coerce')
data['# pulled'] = pd.to_numeric(data['# pulled'].str.replace(',', ''), errors='coerce')
data['percent of total pulls'] = pd.to_numeric(data['percent of total pulls'], errors='coerce')
data['estimated character revenue (in millions)'] = pd.to_numeric(
    data['estimated character revenue (in millions)'].str.replace(r'[\$,]', '', regex=True), errors='coerce'
)
data['time in between first appearence in story and banner (days)'] = pd.to_numeric(
    data['time in between first appearence in story and banner (days)'], errors='coerce'
)
data['Youtube Views'] = pd.to_numeric(data['Youtube Views'].str.replace(',', ''), errors='coerce')
data = data.dropna()
conn = sqlite3.connect("characters_database_with_stats.db")
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS Characters (
    character_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    total_pulls INTEGER,
    pulled INTEGER,
    percent_total_pulls REAL,
    estimated_revenue REAL,
    planet_group TEXT,
    time_between_story_and_banner INTEGER,
    youtube_views INTEGER,
    importance_to_main_story TEXT,
    role TEXT,
    type TEXT,
    gender TEXT,
    path TEXT,
    element TEXT
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS Numerical_Statistics (
    statistic_id INTEGER PRIMARY KEY AUTOINCREMENT,
    column_name TEXT,
    mean REAL,
    median REAL,
    std_dev REAL,
    pearson_corr REAL,
    pearson_p_value REAL,
    spearman_corr REAL,
    spearman_p_value REAL,
    r_squared REAL
)
""")
cursor.execute("""
CREATE TABLE IF NOT EXISTS Categorical_Statistics (
    statistic_id INTEGER PRIMARY KEY AUTOINCREMENT,
    column_name TEXT,
    group_name TEXT,
    group_mean REAL,
    group_median REAL,
    group_std REAL,
    f_statistic REAL,
    p_value REAL
)
""")
for _, row in data.iterrows():
    cursor.execute("""
    INSERT INTO Characters (
        name, total_pulls, pulled, percent_total_pulls, estimated_revenue,
        planet_group, time_between_story_and_banner, youtube_views,
        importance_to_main_story, role, type, gender, path, element
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        row['Character'], row['Total Pulls'], row['# pulled'], row['percent of total pulls'],
        row['estimated character revenue (in millions)'], row['Planet/Group'],
        row['time in between first appearence in story and banner (days)'],
        row['Youtube Views'], row['Importance to Main Story'], row['Role'], row['Type'],
        row['Gender'], row['Path'], row['Element']
    ))
numerical_columns = ['Youtube Views', 'time in between first appearence in story and banner (days)', 
                     '# pulled', 'estimated character revenue (in millions)']

for col in numerical_columns:
    col_mean = data[col].mean()
    col_median = data[col].median()
    col_std = data[col].std()

    pearson_corr, pearson_pval = pearsonr(data['Total Pulls'], data[col])
    spearman_corr, spearman_pval = spearmanr(data['Total Pulls'], data[col])

    X = data[col].values.reshape(-1, 1)
    y = data['Total Pulls'].values
    regressor = LinearRegression()
    regressor.fit(X, y)
    r_squared = regressor.score(X, y)

    cursor.execute("""
    INSERT INTO Numerical_Statistics (
        column_name, mean, median, std_dev, pearson_corr, pearson_p_value, 
        spearman_corr, spearman_p_value, r_squared
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (col, col_mean, col_median, col_std, pearson_corr, pearson_pval, spearman_corr, spearman_pval, r_squared))

categorical_columns = ['Importance to Main Story', 'Role', 'Type', 'Gender', 'Path', 'Element']
for col in categorical_columns:
    groups = data[col].unique()
    for group in groups:
        group_data = data[data[col] == group]['Total Pulls']
        group_mean = group_data.mean()
        group_median = group_data.median()
        group_std = group_data.std()
        anova_groups = [data[data[col] == g]['Total Pulls'] for g in groups]
        if len(anova_groups) > 1:
            f_stat, p_val = f_oneway(*anova_groups)
        else:
            f_stat, p_val = None, None

        cursor.execute("""
        INSERT INTO Categorical_Statistics (
            column_name, group_name, group_mean, group_median, group_std, f_statistic, p_value
        ) VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (col, group, group_mean, group_median, group_std, f_stat, p_val))

conn.commit()
conn.close()
print("Data and statistics have been successfully stored in the SQLite database.")


Data and statistics have been successfully stored in the SQLite database.


In [4]:
###### import csv

def update_percent_total_pulls(cursor):
    cursor.execute("SELECT SUM(total_pulls) FROM Characters")
    total_pulls_sum = cursor.fetchone()[0]
    if total_pulls_sum and total_pulls_sum > 0:
        cursor.execute("""
        UPDATE Characters 
        SET percent_total_pulls = total_pulls * 100.0 / ?
        """, (total_pulls_sum,))

def add_character(conn):
    cursor = conn.cursor()
    print("Enter the details for the new character:")
    name = input("Name: ")
    total_pulls = int(input("Total Pulls: "))
    pulled = int(input("# Pulled: "))
    percent_pulled = 0.0  # Will be updated after the addition
    estimated_revenue = float(input("Estimated Character Revenue (in millions): "))
    planet_group = input("Planet/Group: ")
    time_between = int(input("Time in Between First Appearance and Banner (days): "))
    youtube_views = int(input("Youtube Views: "))
    importance = input("Importance to Main Story (Major/Minor/Not in it): ")
    role = input("Role: ")
    character_type = input("Type: ")
    gender = input("Gender: ")
    path = input("Path: ")
    element = input("Element: ")

    cursor.execute("""
    INSERT INTO Characters (
        name, total_pulls, pulled, percent_total_pulls, estimated_revenue,
        planet_group, time_between_story_and_banner, youtube_views,
        importance_to_main_story, role, type, gender, path, element
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (name, total_pulls, pulled, percent_pulled, estimated_revenue, planet_group,
          time_between, youtube_views, importance, role, character_type, gender, path, element))
    update_percent_total_pulls(cursor)
    conn.commit()
    print("Character added successfully!")

def remove_character(conn):
    cursor = conn.cursor()
    name = input("Enter the name of the character to remove: ")
    cursor.execute("DELETE FROM Characters WHERE name = ?", (name,))
    update_percent_total_pulls(cursor)
    conn.commit()
    print("Character removed successfully!")

def test_character(conn):
    cursor = conn.cursor()
    print("Enter the details for the test character:")
    name = input("Name: ")
    planet_group = input("Planet/Group: ")
    time_between = int(input("Time in Between First Appearance and Banner (days): "))
    youtube_views = int(input("Youtube Views: "))
    importance = input("Importance to Main Story (Major/Minor/Not in it): ")
    role = input("Role: ")
    character_type = input("Type: ")
    gender = input("Gender: ")
    path = input("Path: ")
    element = input("Element: ")

    cursor.execute("SELECT AVG(estimated_revenue / total_pulls) FROM Characters WHERE total_pulls > 0")
    avg_ratio_revenue_to_pulls = cursor.fetchone()[0] or 0.1

    cursor.execute("SELECT AVG(total_pulls / youtube_views) FROM Characters WHERE youtube_views > 0")
    avg_ratio_views_to_pulls = cursor.fetchone()[0] or 0.05
    simulated_total_pulls = max(int(youtube_views * avg_ratio_views_to_pulls), 1000000)
    simulated_pulled = round(simulated_total_pulls / 94)
    simulated_pulled = max(1000, min(simulated_pulled, int(simulated_total_pulls * 0.15)))
    simulated_revenue = round(simulated_total_pulls * avg_ratio_revenue_to_pulls, 2)
    cursor.execute("""
    INSERT INTO Characters (
        name, total_pulls, pulled, percent_total_pulls, estimated_revenue,
        planet_group, time_between_story_and_banner, youtube_views,
        importance_to_main_story, role, type, gender, path, element
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (name, simulated_total_pulls, simulated_pulled, 0.0, simulated_revenue, planet_group,
          time_between, youtube_views, importance, role, character_type, gender, path, element))
    update_percent_total_pulls(cursor)

    print("\nData sorted by Total Pulls:")
    cursor.execute("SELECT * FROM Characters ORDER BY total_pulls DESC")
    rows = cursor.fetchall()
    for row in rows:
        try:
            total_pulls = int(row[2])
            pulled = int(row[3])
            percent_total_pulls = float(row[4])
            estimated_revenue = float(row[5])  
            time_between_db = int(row[7])
            youtube_views_db = int(row[8])

            print(f"""
                Character: {row[1]}
                Total Pulls: {total_pulls}
                # Pulled: {pulled}
                % of Total Pulls: {percent_total_pulls:.2f}%
                Estimated Revenue (in millions): ${estimated_revenue:.2f}
                Planet/Group: {row[6]}
                Time Between Story and Banner (days): {time_between_db}
                Youtube Views: {youtube_views_db}
                Importance to Main Story: {row[9]}
                Role: {row[10]}
                Type: {row[11]}
                Gender: {row[12]}
                Path: {row[13]}
                Element: {row[14]}
            """)
        except ValueError as e:
            print(f"Error displaying character {row[1]}: {e}")

    cursor.execute("DELETE FROM Characters WHERE name = ?", (name,))
    update_percent_total_pulls(cursor)
    conn.commit()
    print("\nTest character removed.")



def print_csv(conn):
    import pandas as pd
    input_file = "HSR.csv"
    original_data = pd.read_csv(input_file)
    last_row = original_data.iloc[-1].to_dict()  
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Characters ORDER BY name")
    rows = cursor.fetchall()
    cursor.execute("SELECT name FROM Characters WHERE name = ?", (last_row["Character"],))
    reruns_exists = cursor.fetchone() is not None

    if not reruns_exists:
        cursor.execute("""
        INSERT INTO Characters (
            name, total_pulls, pulled, percent_total_pulls, estimated_revenue,
            planet_group, time_between_story_and_banner, youtube_views,
            importance_to_main_story, role, type, gender, path, element
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            last_row["Character"], 
            int(str(last_row["Total Pulls"]).replace(",", "")),  
            int(str(last_row["# pulled"]).replace(",", "")),  
            float(last_row["percent of total pulls"]),  
            float(str(last_row["estimated character revenue (in millions)"]).replace("$", "").replace(",", "")),  
            last_row["Planet/Group"], 
            int(last_row["time in between first appearence in story and banner (days)"]) if pd.notna(last_row["time in between first appearence in story and banner (days)"]) else 0,
            int(float(last_row["Youtube Views"])) if pd.notna(last_row["Youtube Views"]) else 0, 
            last_row["Importance to Main Story"], 
            last_row["Role"],
            last_row["Type"], 
            last_row["Gender"], 
            last_row["Path"], 
            last_row["Element"]
        ))
        conn.commit()
    
    cursor.execute("SELECT SUM(total_pulls) FROM Characters")
    total_pulls_sum = cursor.fetchone()[0]

    cursor.execute("SELECT * FROM Characters ORDER BY name != 'Reruns', name")
    rows = cursor.fetchall()

    updated_rows = []
    for row in rows:
        total_pulls = int(str(row[2]).replace(",", "")) 
        percent_total_pulls = (total_pulls * 100) / total_pulls_sum if total_pulls_sum > 0 else 0
        updated_row = list(row)
        updated_row[4] = round(percent_total_pulls, 3)  
        updated_rows.append(updated_row)

    reruns_row = next((row for row in updated_rows if row[1] == 'Reruns'), None)
    if reruns_row:
        updated_rows = [row for row in updated_rows if row[1] != 'Reruns']
        updated_rows.append(reruns_row)

    output_file = "New_HSR.csv"
    headers = [
        "Character", "Total Pulls", "# pulled", "percent of total pulls",
        "estimated character revenue (in millions)", "Planet/Group",
        "time in between first appearence in story and banner (days)", "Youtube Views",
        "Importance to Main Story", "Role", "Type", "Gender", "Path", "Element"
    ]

    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for row in updated_rows:
            writer.writerow(row[1:])  # Skip the character_id field

    print(f"CSV file '{output_file}' has been created successfully!")



def view_sorted_data(conn):
    cursor = conn.cursor()
    print("\nChoose a column to sort by:")
    print("1. Total Pulls")
    print("2. # Pulled")
    print("3. Estimated Character Revenue (in millions)")
    print("4. Time in Between First Appearance and Banner (days)")
    print("5. Youtube Views")
    choice = input("Enter your choice (1-5): ")

    columns = {
        "1": "total_pulls",
        "2": "pulled",
        "3": "estimated_revenue",
        "4": "time_between_story_and_banner",
        "5": "youtube_views"
    }

    if choice in columns:
        column = columns[choice]
        cursor.execute(f"SELECT * FROM Characters ORDER BY {column} DESC")
        rows = cursor.fetchall()
        print(f"\nData sorted by {column.replace('_', ' ').title()}:")
        for row in rows:
            total_pulls = int(str(row[2]).replace(",", "")) if row[2] else 0
            pulled = int(str(row[3]).replace(",", "")) if row[3] else 0
            percent_total_pulls = float(row[4]) if row[4] else 0.0
            estimated_revenue = (
                float(str(row[5]).replace("$", "").replace(",", "")) if row[5] else 0.0
            )

            time_between = int(row[7]) if row[7] else 0
            youtube_views = int(str(row[8]).replace(",", "")) if row[8] else 0

            print(f"""
                Character: {row[1]}
                Total Pulls: {total_pulls}
                # Pulled: {pulled}
                % of Total Pulls: {percent_total_pulls:.2f}%
                Estimated Revenue (in millions): ${estimated_revenue:.2f}
                Planet/Group: {row[6]}
                Time Between Story and Banner (days): {time_between}
                Youtube Views: {youtube_views}
                Importance to Main Story: {row[9]}
                Role: {row[10]}
                Type: {row[11]}
                Gender: {row[12]}
                Path: {row[13]}
                Element: {row[14]}
            """)
    else:
        print("Invalid choice!")

def remove_duplicates(cursor):
    cursor.execute("""
    DELETE FROM Characters
    WHERE rowid NOT IN (
        SELECT MAX(rowid)
        FROM Characters
        GROUP BY name
    )
    """)
    print("Duplicates removed successfully!")

def main():
    conn = sqlite3.connect("characters_database_with_stats.db")
    cursor = conn.cursor()

    while True:
        remove_duplicates(cursor)
        update_percent_total_pulls(cursor)
        conn.commit()

        print("\nMenu:")
        print("1. Add a Character")
        print("2. Remove a Character")
        print("3. Test a Character")
        print("4. View Sorted Data")
        print("5. Print CSV")
        print("6. Exit")
        choice = input("Enter your choice (1-6): ")

        if choice == "1":
            add_character(conn)
        elif choice == "2":
            remove_character(conn)
        elif choice == "3":
            test_character(conn)
        elif choice == "4":
            view_sorted_data(conn)
        elif choice == "5":
            print_csv(conn)
        elif choice == "6":
            conn.close()
            print("Goodbye!")
            break
        else:
            print("Invalid choice! Please try again.")


if __name__ == "__main__":
    main()


Duplicates removed successfully!

Menu:
1. Add a Character
2. Remove a Character
3. Test a Character
4. View Sorted Data
5. Print CSV
6. Exit


Enter your choice (1-6):  4



Choose a column to sort by:
1. Total Pulls
2. # Pulled
3. Estimated Character Revenue (in millions)
4. Time in Between First Appearance and Banner (days)
5. Youtube Views


Enter your choice (1-5):  1



Data sorted by Total Pulls:

                Character: Reruns
                Total Pulls: 81209986
                # Pulled: 854299
                % of Total Pulls: 15.66%
                Estimated Revenue (in millions): $170.15
                Planet/Group: None
                Time Between Story and Banner (days): 0
                Youtube Views: 0
                Importance to Main Story: None
                Role: None
                Type: None
                Gender: None
                Path: None
                Element: None
            

                Character: Acheron
                Total Pulls: 45381074
                # Pulled: 526623
                % of Total Pulls: 8.75%
                Estimated Revenue (in millions): $95.08
                Planet/Group: GalaxyRanger
                Time Between Story and Banner (days): 49
                Youtube Views: 4583018
                Importance to Main Story: Major
                Role: Damage Dealer
                T

Enter your choice (1-6):  6


Goodbye!
